In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산_20250731.csv')

df['price'] = df['price'].str.replace('가격협의', '-2')
df['price'] = df['price'].str.replace('(1개월단위)', '')
df['price'] = df['price'].str.split('~').str[0]
df['price'] = df['price'].fillna('-1')
df['price'] = df['price'].astype(np.int64)

df

In [ ]:
df['data_size'] = '-'

In [ ]:
import os
from bs4 import BeautifulSoup

base_dir = 'KDX'
i = 0
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        with open(file_path, 'r') as f:
            html = f.read()
        soup = BeautifulSoup(html, 'html.parser')

        row = soup.find('th', string='용량')
        if row:
            data_size = row.find_next_sibling('td').get_text(strip=True)
            print(i, file_path, data_size)
            df.loc[df['filename'] == filename, 'data_size'] = data_size
            i += 1

In [ ]:
df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250806.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산_20250806.csv')

df

In [ ]:
df[df['source'] == 'bigdata-transportation']

In [ ]:
import os
from bs4 import BeautifulSoup

base_dir = '/home/ubuntu/data_value/bigdata-transportation-html'

i = 0
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        with open(file_path, 'r') as f:
            html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        
        if soup:
            try:
                data_size = soup.find('td', id='file-size').get_text(strip=True)
            except Exception as e:
                data_size = '-2'
            if data_size == '':
                data_size = '-1'
            
            print(i, file_path, data_size)
            df.loc[df['filename'] == f'definition_{filename.split('.')[0]}.xlsx', 'data_size'] = data_size
            i += 1

In [ ]:
df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250806_2.csv', index=False)

In [ ]:
import pandas as pd
import re

df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산_20250806_4.csv', dtype=str)

# 1. 변환 함수
def to_bytes(text):
    if not isinstance(text, str):
        return None

    text = text.replace(',', '').strip().upper()

    multiplier = 1
    if 'KB' in text:
        multiplier = 1024
        text = text.replace('KB', '')
    elif 'MB' in text:
        multiplier = 1024 * 1024
        text = text.replace('MB', '')
    elif 'GB' in text:
        multiplier = 1024 * 1024 * 1024
        text = text.replace('GB', '')

    match = re.search(r'[\d.]+', text)
    if not match:
        return None

    value = float(match.group(0))
    return int(value * multiplier)

# 3. 변환 적용
df['data_size_bytes'] = df['data_size'].apply(to_bytes)

# 4. 결과 확인
df


In [ ]:
df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250806_5.csv', index=False)